>### 🚩 *Create a free WhyLabs account to get more value out of whylogs!*<br> 
>*Did you know you can store, visualize, and monitor whylogs profiles with the [WhyLabs Observability Platform](https://whylabs.ai/whylogs-free-signup?utm_source=whylogs-Github&utm_medium=whylogs-example&utm_campaign=Writing_Feature_Weights_to_WhyLabs)? Sign up for a [free WhyLabs account](https://whylabs.ai/whylogs-free-signup?utm_source=whylogs-Github&utm_medium=whylogs-example&utm_campaign=Writing_Feature_Weights_to_WhyLabs) to leverage the power of whylogs and WhyLabs together!*

# Feature Importance with WhyLabs

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/whylabs/whylogs/blob/mainline/python/examples/integrations/writers/Writing_Feature_Weights_to_WhyLabs.ipynb)

Feature Importance consists of a class of techniques that indicates the relative importance of each feature for a given prediction model. This is done by calculating and assigning a score, or weight, to each feature. Tracking the feature weights of a model can be useful for different reasons, such as:

- Data Insights: By determining which features are more or less important to your task, a domain expert can make informed decisions, such as gathering more data or adding/removing different features.
- Model Interpretability: Feature weights can provide insights into which features your model considers more or less relevant, improving the model's interpretability.
- Model Upgrading: Feature weights can be the basis of further feature selection processes, removing unnecessary features, improving the maintainability and performance of your model.

WhyLabs supports feature weight tracking and visualization at your monitoring dashboard. The most straightforward way to send and receive feature weights to WhyLabs is through whylogs, and that's exactly what we'll show in this example.

In this example, we will:

- Calculate simple feature weights with Linear Regression
- Send feature weights to your dashboard at WhyLabs
- Get latest version of feature weights from your dashboard

## Installing dependencies

First, let's make sure you have the required packages installed:

In [ ]:
# Note: you may need to restart the kernel to use updated packages.
%pip install whylogs[whylabs]
%pip install scikit-learn==1.0.2

## Calculating Feature Weights

There are several different ways of calculating feature importance. One such way is calculating the model coefficients of a linear regression model, which can be interpreted as a feature importance score. That's the method we'll use for this demonstration. We'll create 5 informative features and 5 random ones.

The code below was based on the article [How to Calculate Feature Importance With Python](https://machinelearningmastery.com/calculate-feature-importance-with-python/), by Jason Brownlee. I definitely recommend it if you are interested in other ways of calculating feature importance.

In [5]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
# define dataset
X, y = make_regression(n_samples=1000, n_features=10, n_informative=5, random_state=1)
# define the model
model = LinearRegression()
# fit the model
model.fit(X, y)
# get importance
importance = model.coef_
# summarize feature importance

weights = {"Feature_{}".format(key): value for (key, value) in enumerate(importance)}

weights

{'Feature_0': -3.330399790430435e-15,
 'Feature_1': 12.44482785538977,
 'Feature_2': -4.393883142916863e-14,
 'Feature_3': -2.7047779443616894e-14,
 'Feature_4': 93.32225450776932,
 'Feature_5': 86.50810998606804,
 'Feature_6': 26.746066698034504,
 'Feature_7': 3.285346398262185,
 'Feature_8': -2.7795267559640957e-14,
 'Feature_9': 3.430594380756143e-14}

We end up with a dictionary with the features as keys and the respective scores as values. This is an example of *global* feature importance, as opposed to *local* feature importance, which would show the contribution of features for a specific prediction. Currently, WhyLabs and whylogs support only global feature importance. Therefore, this is the structure we'll use to later send the Feature Weights to WhyLabs.

## ✔️ Setting the Environment Variables

In order to send our profile to WhyLabs, let's first set up an account. You can skip this if you already have an account and a model set up.

We will need three pieces of information:

- API token
- Organization ID
- Dataset ID (or model-id)

First, grab a [free WhyLabs account](https://whylabs.ai/whylogs-free-signup?utm_source=whylogs-Github&utm_medium=whylogs-example&utm_campaign=Writing_Feature_Weights_to_WhyLabs) if you haven't already. You can follow along with the examples if you wish, but if you’re interested in only following this demonstration, you can go ahead and skip the quick start instructions.

After that, you’ll be prompted to create an API token. Once you create it, copy and store it locally. The second important information here is your org ID. Take note of it as well. After you get your API Token and Org ID, you can go to https://hub.whylabsapp.com/models to see your projects dashboard. You can create a new project and take note of it's ID (if it's a model project it will look like `model-xxxx`).

In [6]:
import getpass
import os

# set your org-id here - should be something like "org-xxxx"
print("Enter your WhyLabs Org ID") 
os.environ["WHYLABS_DEFAULT_ORG_ID"] = input()

# set your datased_id (or model_id) here - should be something like "model-xxxx"
print("Enter your WhyLabs Dataset ID")
os.environ["WHYLABS_DEFAULT_DATASET_ID"] = input()

# set your API key here
print("Enter your WhyLabs API key")
os.environ["WHYLABS_API_KEY"] = getpass.getpass()

print("Using API Key ID: ", os.environ["WHYLABS_API_KEY"][0:10])

## Writing Feature Weights to WhyLabs

Once the feature weights are calculated, sending them to WhyLabs is very simple.

We first need to wrap the dictionary into a `FeatureWeights` object:

In [7]:
from whylogs.core.feature_weights import FeatureWeights
feature_weights = FeatureWeights(weights)

And then use the `WhyLabsWriter` to write it, provided your environment variables are properly set: 

In [8]:
from whylogs.api.writer.whylabs import WhyLabsWriter
result = feature_weights.writer("whylabs").write()

result

(True, '200')

Another way of doing the exact same thing is by instantiating the Writer itself, and then calling `write`:

In [13]:
WhyLabsWriter().write(feature_weights)

(True, '200')

## Getting Feature Weights from WhyLabs

You can also get the feature weights from WhyLabs with `get_feature_weights()`:

In [14]:
result = WhyLabsWriter().get_feature_weights()

print(result.weights)
print(result.metadata)

{'Feature_4': 93.32225450776932, 'Feature_5': 86.50810998606804, 'Feature_6': 26.746066698034504, 'Feature_1': 12.44482785538977, 'Feature_7': 3.285346398262185, 'Feature_2': -4.393883142916863e-14, 'Feature_9': 3.430594380756143e-14, 'Feature_8': -2.7795267559640957e-14, 'Feature_3': -2.7047779443616894e-14, 'Feature_0': -3.330399790430435e-15}
{'version': 28, 'updatedTimestamp': 1664373654215, 'author': 'system'}


As you can see, the result will contain the set of weights in `result.weights`, along with additional metadata in `result.metadata`.

If you write multiple set of weights to the same model at WhyLabs, the content will be overwritten. When using `get_feature_weights()`, you'll get the latest version, that is, the last set of weights you sent. You're able to see which version it is in the metadata, along with the timestamp of creation.

## Reference

- [Understanding Feature Importance and How to Implement it in Python](https://towardsdatascience.com/understanding-feature-importance-and-how-to-implement-it-in-python-ff0287b20285)
- [How to Calculate Feature Importance With Python](https://machinelearningmastery.com/calculate-feature-importance-with-python/)
- [Feature importance — what’s in a name?](https://medium.com/bigdatarepublic/feature-importance-whats-in-a-name-79532e59eea3)